## AMP FOR SPIKED-TENSOR-MODELS

This is an implementation of the AMP algorithm for the mixed spiked tensor model (2+3), as written in https://arxiv.org/pdf/1812.09066.pdf written in pytorch. While it runs on CPU, porting it on GPU is trivial.

Here is the algorithm, written with a Gaussian prior

In [5]:
import torch
from math import sqrt

# Priors
def prior_gb(A, B, prmts):
    """Compute f_a and f_c for Gauss-Bernoulli prior"""

    rho, mu, sig = prmts

    m = (B * sig + mu) / (1. + A * sig)
    v = sig / (1 + A * sig)
    keep = (rho + (1 - rho) * torch.sqrt(1. + A * sig) *
            torch.exp(-.5 * m ** 2 / v + .5 * mu ** 2 / sig))
    p_s = rho / keep

    a = p_s * m
    c = p_s * v + p_s * (1. - p_s) * m ** 2
    return a, torch.mean(c)


# AMP Solver
def amp(Y, T, Delta2, Delta3,
        prior=prior_gb, prior_prmts=None, true_coef=None,
        max_iter=250, tol=1e-13, verbose=1):
    """Iterate VAMP equations"""

    N, _ = Y.shape

    # Initialize variables
    B2 = torch.zeros(N, 1)
    A2 = 0.
    B3 = torch.zeros(N, 1)
    A3 = 0.
    a = torch.randn(N, 1)
    a_old = torch.zeros(N, 1)
    c = 1.
    S2 = sqrt(2)
    SN = sqrt(N)
    K = S2 / (Delta3 * N)
    if verbose > 0:
        print("time ; convergence mse, Matrix-mmse ")
    for t in range(max_iter):
        # Messages/estimates on x from likelihood
        B2 = (1. / (SN * Delta2)) * Y @ a - (c / Delta2) * a_old
        A2 = (1. / Delta2) * torch.mean(a * a)
        B3 = (K * torch.transpose(a, 0, 1)@T@a).view(N, -1)
        B3 = B3 - (2 / Delta3) * c * (torch.mean(a * a_old)) * a_old
        A3 = (1. / Delta3) * torch.mean(a * a)**2

        a_old = torch.clone(a)
        a, c = prior(A2 + A3, B2 + B3, prior_prmts)

        a = 0.5 * a + 0.5 * a_old

        # Compute metrics
        conv = torch.mean((a - a_old) ** 2)
        mse = torch.mean((a - true_coef) ** 2) if true_coef is not None else 0.
        Mmse = torch.mean((a@torch.transpose(a, 0, 1) - (true_coef@torch.transpose(true_coef, 0, 1)))**2) if true_coef is not None else 0.
        if verbose > 0:
            print("t = %d; conv = %g, mse = %g, Mmse = %g" % (t, conv, mse, Mmse))
        if conv < tol:
            break
    return a

Now, this is a short demo on how the algorithm works:

In [6]:
import torch
# We keep CPU by default, if you want GPU uncomment the next line
# torch.set_default_tensor_type('torch.cuda.FloatTensor')#GPU by default
from math import sqrt
import time

N = 100
DELTA2 = 0.01
DELTA3 = 1

X0 = torch.randn(N, 1)
Y = X0@torch.transpose(X0, 0, 1)
T = Y.view(N, N, 1)@torch.transpose(X0, 0, 1)

M_N = torch.randn(N, N)
Matrix_noise = (M_N + torch.transpose(M_N, 0, 1))/sqrt(2)
Y = Y / sqrt(N) + Matrix_noise * sqrt(DELTA2)

T_N = torch.randn(N, N, N)
R1 = torch.transpose(T_N,0,1)
R2 = torch.transpose(T_N,0,2)
R3 = torch.transpose(T_N,1,2)
R4 = torch.transpose(R1,1,2)
R5 = torch.transpose(R2,1,2)
Tensor_noise = (T_N + R1 + R2 + R3 + R4 +R5)/sqrt(6)
T = (sqrt(2) / N) * T + Tensor_noise * sqrt(DELTA3)

t = time.time()
Xhat = amp(Y, T, DELTA2, DELTA3,
           prior=prior_gb, prior_prmts=(1, 0, 1),
           true_coef=X0, max_iter=250)
elapsed = time.time() - t

time ; convergence mse, Matrix-mmse 
t = 0; conv = 0.241033, mse = 1.15033, Mmse = 1.28858
t = 1; conv = 0.106367, mse = 0.682179, Mmse = 1.11205
t = 2; conv = 0.934296, mse = 0.0646466, Mmse = 0.252066
t = 3; conv = 0.0312509, mse = 0.0166379, Mmse = 0.0378277
t = 4; conv = 0.00132645, mse = 0.0117742, Mmse = 0.0265073
t = 5; conv = 0.000400079, mse = 0.0102972, Mmse = 0.0232327
t = 6; conv = 0.000122177, mse = 0.00982795, Mmse = 0.0221993
t = 7; conv = 3.7669e-05, mse = 0.00967358, Mmse = 0.021859
t = 8; conv = 1.17203e-05, mse = 0.00962093, Mmse = 0.0217426
t = 9; conv = 3.67074e-06, mse = 0.00960216, Mmse = 0.0217009
t = 10; conv = 1.15545e-06, mse = 0.00959509, Mmse = 0.0216851
t = 11; conv = 3.65202e-07, mse = 0.00959226, Mmse = 0.0216787
t = 12; conv = 1.15844e-07, mse = 0.00959103, Mmse = 0.0216759
t = 13; conv = 3.68574e-08, mse = 0.00959046, Mmse = 0.0216746
t = 14; conv = 1.17605e-08, mse = 0.00959019, Mmse = 0.0216739
t = 15; conv = 3.76181e-09, mse = 0.00959004, Mmse = 0.0